In [5]:
# Cell 1: Import necessary libraries
import tensorflow as tf
import sionna
import numpy as np
import matplotlib.pyplot as plt
from sionna.ofdm import ResourceGrid
from sionna.mimo import StreamManagement
from sionna.channel.tr38901 import CDL, AntennaArray
from sionna.utils import BinarySource, ebnodb2no
from sionna.fec.ldpc.encoding import LDPC5GEncoder
from sionna.mapping import Mapper
from sionna.ofdm import ResourceGridMapper
from sionna.channel import subcarrier_frequencies, cir_to_ofdm_channel, ApplyOFDMChannel

In [13]:
# Cell 2: Define setup_system function
def setup_system(num_ut=1, 
                 num_bs=1, 
                 num_ut_ant=1, 
                 num_bs_ant=1, 
                 carrier_frequency=2.6e9,
                 delay_spread=300e-9, 
                 cdl_model="B", 
                 speed=10.0, 
                 direction="uplink"):
    
    num_streams_per_tx = num_ut_ant
    rx_tx_association = np.array([[1]])

    sm = StreamManagement(rx_tx_association, num_streams_per_tx)

    rg = ResourceGrid(
        num_ofdm_symbols=14,
        fft_size=76,
        subcarrier_spacing=15e3,
        num_tx=1,
        num_streams_per_tx=num_streams_per_tx,
        cyclic_prefix_length=6,
        num_guard_carriers=[6, 5],
        dc_null=True,
        pilot_pattern="kronecker",
        pilot_ofdm_symbol_indices=[2, 11]
    )

    ut_array = AntennaArray(
        num_rows=1,
        num_cols=num_ut_ant // 2,
        polarization="single",
        polarization_type="V",
        antenna_pattern="38.901",
        carrier_frequency=carrier_frequency
    )
    bs_array = AntennaArray(
        num_rows=1,
        num_cols=num_bs_ant // 2,
        polarization="single",
        polarization_type="V",
        antenna_pattern="38.901",
        carrier_frequency=carrier_frequency
    )

    cdl = CDL(
        model=cdl_model,
        delay_spread=delay_spread,
        carrier_frequency=carrier_frequency,
        ut_array=ut_array,
        bs_array=bs_array,
        direction=direction,
        min_speed=speed
    )

    print("System Setup: Complete")
    print(f"FFT size: {rg.fft_size}, Subcarrier spacing: {rg.subcarrier_spacing/1e3:.1f} kHz")
    print(f"Effective subcarriers: {rg.fft_size - sum(rg.num_guard_carriers) - int(rg.dc_null)}")
    print(f"Tx antennas (UT): {num_ut_ant}, Rx antennas (BS): {num_bs_ant}")
    print(f"Channel: CDL-{cdl_model}, Delay spread: {delay_spread*1e9:.0f} ns, Speed: {speed} m/s")

    return sm, rg, cdl

In [15]:
# Cell 3: Define generate_signal function (modified to return x_rg for no PA)
def generate_signal(rg, batch_size, num_bits_per_symbol, coderate):
    n = int(rg.num_data_symbols * num_bits_per_symbol)
    k = int(n * coderate)

    binary_source = BinarySource()
    encoder = LDPC5GEncoder(k, n)
    mapper = Mapper("qam", num_bits_per_symbol)
    rg_mapper = ResourceGridMapper(rg)

    b = binary_source([batch_size, 1, rg.num_streams_per_tx, k])
    c = encoder(b)
    x = mapper(c)
    x_rg = rg_mapper(x)
    z_target = x

    x_time_freq = tf.signal.ifft(tf.cast(x_rg, tf.complex64))
    x_time = tf.reshape(x_time_freq[:, 0, :, :, :], [batch_size, rg.num_streams_per_tx, -1])
    x_time_ri = tf.stack([tf.math.real(x_time), tf.math.imag(x_time)], axis=-1)

    print("Signal Generation: Complete")
    print(f"Bits shape: {b.shape}, QAM shape: {x.shape}, Time-domain shape: {x_time_ri.shape}")

    return x_time_ri, z_target, b, x_rg

In [16]:
# Cell 4: Define apply_channel function
def apply_channel(x_pa, rg, cdl, ebno_db, num_bits_per_symbol, coderate):
    frequencies = subcarrier_frequencies(rg.fft_size, rg.subcarrier_spacing)
    cir = cdl(batch_size=x_pa.shape[0], num_time_steps=rg.num_ofdm_symbols,
              sampling_frequency=rg.fft_size * rg.subcarrier_spacing)
    h_freq = cir_to_ofdm_channel(frequencies, *cir, normalize=True)

    no = ebnodb2no(ebno_db, num_bits_per_symbol, coderate, rg)
    ofdm_channel = ApplyOFDMChannel(add_awgn=True)
    y_rc = ofdm_channel([x_pa, h_freq, no])

    y_td = tf.signal.ifft(tf.cast(y_rc, tf.complex64))
    y_flat = tf.reshape(y_td[:, 0, :, :, :], [y_rc.shape[0], y_rc.shape[2], -1])
    y_flat = tf.transpose(y_flat, [0, 2, 1])
    x_input_rc1 = tf.stack([tf.math.real(y_flat), tf.math.imag(y_flat)], axis=-1)

    print("Channel Application: Complete")
    print(f"Channel shape: {h_freq.shape}, RC input shape: {x_input_rc1.shape}")

    return x_input_rc1, h_freq

In [17]:
# Cell 5: Define prepare_data function (this creates the data for RC model)
def prepare_data(x_input_rc1, x_time_ri, z_target, val_fraction=0.25):
    batch_size = x_input_rc1.shape[0]
    num_val = int(batch_size * val_fraction)
    num_train = batch_size - num_val

    train_idx = slice(0, num_train)
    val_idx = slice(num_train, batch_size)
    x_in_train = x_input_rc1[train_idx]
    x_in_val = x_input_rc1[val_idx]
    teacher_td = tf.transpose(x_time_ri, [0, 2, 1, 3])
    teacher_td_train = teacher_td[train_idx]
    teacher_td_val = teacher_td[val_idx]
    z_target_r = tf.reshape(z_target, [batch_size, -1, z_target.shape[-1]])
    z_target_train = z_target_r[train_idx]
    z_target_val = z_target_r[val_idx]

    mean_train = tf.reduce_mean(x_in_train)
    std_train = tf.math.reduce_std(x_in_train) + 1e-6
    x_in_train = (x_in_train - mean_train) / std_train
    mean_val = tf.reduce_mean(x_in_val)
    std_val = tf.math.reduce_std(x_in_val) + 1e-6
    x_in_val = (x_in_val - mean_val) / std_val

    train_ds = tf.data.Dataset.from_tensor_slices(
        (x_in_train, teacher_td_train, z_target_train)
    ).shuffle(buffer_size=num_train).batch(16)
    val_ds = tf.data.Dataset.from_tensor_slices(
        (x_in_val, teacher_td_val, z_target_val)
    ).batch(num_val)

    print("Data Preparation: Complete")
    print(f"Train frames: {num_train}, Validation frames: {num_val}")
    print(f"Train input shape: {x_in_train.shape}")

    return train_ds, val_ds

In [18]:
# Cell 6: Run the pipeline to create data for RC model (without power amplification)
sionna.config.seed = 42

batch_size = 64
num_bits_per_symbol = 2
coderate = 0.5
ebno_db = 10.0
num_ofdm_symbols = 14

# Setup system
sm, rg, cdl = setup_system()

# Generate signal (now returns x_rg)
x_time_ri, z_target, b, x_rg = generate_signal(rg, batch_size, num_bits_per_symbol, coderate)


x_pa = x_rg

# Apply channel
x_input_rc1, h_freq = apply_channel(x_pa, rg, cdl, ebno_db, num_bits_per_symbol, coderate)

# Prepare data for RC (this is the final step for data creation)
train_ds, val_ds = prepare_data(x_input_rc1, x_time_ri, z_target)

# You can now use train_ds and val_ds for RC model training
# For example, print a sample from train_ds
for x, td, z in train_ds.take(1):
    print(f"Sample input shape: {x.shape}")
    print(f"Sample teacher shape: {td.shape}")
    print(f"Sample target shape: {z.shape}")

System Setup: Complete
FFT size: 76, Subcarrier spacing: 15.0 kHz
Effective subcarriers: 64
Tx antennas (UT): 1, Rx antennas (BS): 1
Channel: CDL-B, Delay spread: 300 ns, Speed: 10.0 m/s
Signal Generation: Complete
Bits shape: (64, 1, 1, 768), QAM shape: (64, 1, 1, 768), Time-domain shape: (64, 1, 1064, 2)
Channel Application: Complete
Channel shape: (64, 1, 0, 1, 0, 14, 76), RC input shape: (64, 1064, 0, 2)
Data Preparation: Complete
Train frames: 48, Validation frames: 16
Train input shape: (48, 1064, 0, 2)
Sample input shape: (16, 1064, 0, 2)
Sample teacher shape: (16, 1064, 1, 2)
Sample target shape: (16, 1, 768)
